In [ ]:
import torch
from abc import ABC, abstractmethod


def calc_out_shape(input_matrix_shape, out_channels, kernel_size, stride, padding):
    batch_size, channels_count, input_height, input_width = input_matrix_shape
    output_height = (input_height + 2 * padding - (kernel_size - 1) - 1) // stride + 1
    output_width = (input_width + 2 * padding - (kernel_size - 1) - 1) // stride + 1

    return batch_size, out_channels, output_height, output_width


class ABCConv2d(ABC):
    def __init__(self, in_channels, out_channels, kernel_size, stride):
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.kernel_size = kernel_size
        self.stride = stride

    def set_kernel(self, kernel):
        self.kernel = kernel

    @abstractmethod
    def __call__(self, input_tensor):
        pass


class Conv2d(ABCConv2d):
    def __init__(self, in_channels, out_channels, kernel_size, stride):
        self.conv2d = torch.nn.Conv2d(in_channels, out_channels, kernel_size,
                                      stride, padding=0, bias=False)

    def set_kernel(self, kernel):
        self.conv2d.weight.data = kernel

    def __call__(self, input_tensor):
        return self.conv2d(input_tensor)


def create_and_call_conv2d_layer(conv2d_layer_class, stride, kernel, input_matrix):
    out_channels = kernel.shape[0]
    in_channels = kernel.shape[1]
    kernel_size = kernel.shape[2]

    layer = conv2d_layer_class(in_channels, out_channels, kernel_size, stride)
    layer.set_kernel(kernel)

    return layer(input_matrix)


def test_conv2d_layer(conv2d_layer_class, batch_size=2,
                      input_height=4, input_width=4, stride=2):
    kernel = torch.tensor(
                      [[[[0., 1, 0],
                         [1,  2, 1],
                         [0,  1, 0]],

                        [[1, 2, 1],
                         [0, 3, 3],
                         [0, 1, 10]],

                        [[10, 11, 12],
                         [13, 14, 15],
                         [16, 17, 18]]]])

    in_channels = kernel.shape[1]

    input_tensor = torch.arange(0, batch_size * in_channels *
                                input_height * input_width,
                                out=torch.FloatTensor()) \
        .reshape(batch_size, in_channels, input_height, input_width)

    custom_conv2d_out = create_and_call_conv2d_layer(
        conv2d_layer_class, stride, kernel, input_tensor)
    conv2d_out = create_and_call_conv2d_layer(
        Conv2d, stride, kernel, input_tensor)
    # print("conv2d_out: \n", conv2d_out)

    return torch.allclose(custom_conv2d_out, conv2d_out) \
              and (custom_conv2d_out.shape == conv2d_out.shape)


class Conv2dMatrix(ABCConv2d):
    # Функция преобразование кернела в матрицу нужного вида.
    def _unsqueeze_kernel(self, torch_input, output_height, output_width):
        kernel_unsqueezed = torch.nn.functional.pad(torch_input, (0, 1, 0, 1), # !!!!Установка padding приведена для частного случая stride=2 
                                                    "constant", 0).flatten(start_dim=1)
        # print("kernel_unsqueezed.shape:", kernel_unsqueezed.shape)
        # print("kernel_unsqueezed: \n", kernel_unsqueezed)
        return kernel_unsqueezed

    def __call__(self, torch_input):
        batch_size, out_channels, output_height, output_width\
            = calc_out_shape(
                input_matrix_shape=torch_input.shape,
                out_channels=self.kernel.shape[0],
                kernel_size=self.kernel.shape[2],
                stride=self.stride,
                padding=0)

        kernel_unsqueezed = self._unsqueeze_kernel(self.kernel,\
                                                   output_height,\
                                                    output_width)
        result = kernel_unsqueezed @ torch_input.view((batch_size, -1)).permute(1, 0)
        # print("torch_input.view((batch_size, -1)).shape \n", torch_input.view((batch_size, -1)).shape)
        # print("torch_input.view((batch_size, -1)) \n", torch_input.view((batch_size, -1)))
        # print("torch_input.view((batch_size, -1)).permute(1, 0) \n", torch_input.view((batch_size, -1)).permute(1, 0))
        # print("result.shape:", result.shape)
        # print("result: \n", result)
        # print("result.permute(1, 0).shape:", result.permute(1, 0).shape)
        # print("result.permute(1, 0): \n", result.permute(1, 0))
        return result.permute(1, 0).view((batch_size, self.out_channels,
                                           output_height, output_width))

# Проверка происходит автоматически вызовом следующего кода
# (раскомментируйте для самостоятельной проверки,
#  в коде для сдачи задания должно быть закомментировано):
# print(test_conv2d_layer(Conv2dMatrix))

True


In [39]:
kernel = torch.tensor(
                      [[[[0., 1, 0],
                         [1,  2, 1],
                         [0,  1, 0]],

                        [[1, 2, 1],
                         [0, 3, 3],
                         [0, 1, 10]],

                        [[10, 11, 12],
                         [13, 14, 15],
                         [16, 17, 18]]]])

in_channels = kernel.shape[1]

# input_tensor = torch.arange(0, 2 * in_channels *
#                                 4 * 4,
#                                 out=torch.FloatTensor()) \
#         .reshape(2, in_channels, 4, 4)
print(kernel.shape)
# print(input_tensor.shape)

torch.Size([1, 3, 3, 3])


In [ ]:
kernel_unsqueezed =\
    torch.nn.functional.pad(input=kernel,\
                            pad=(0, 1, 0, 1),\
                            mode='constant', 
                            value=0)

# kernel_unsqueezed = kernel_unsqueezed[:,:, 0*2:0*2+4, 0*2:0*2+4]

print(kernel_unsqueezed.shape)
print(kernel_unsqueezed)

kernel_unsqueezed = torch.flatten(kernel_unsqueezed, start_dim=1)
print(kernel_unsqueezed.shape)
print(kernel_unsqueezed)

kernel_unsqueezed = torch.unsqueeze(kernel_unsqueezed, 0)
print(kernel_unsqueezed.shape)
print(kernel_unsqueezed)

torch.Size([1, 3, 4, 4])
tensor([[[[ 0.,  1.,  0.,  0.],
          [ 1.,  2.,  1.,  0.],
          [ 0.,  1.,  0.,  0.],
          [ 0.,  0.,  0.,  0.]],

         [[ 1.,  2.,  1.,  0.],
          [ 0.,  3.,  3.,  0.],
          [ 0.,  1., 10.,  0.],
          [ 0.,  0.,  0.,  0.]],

         [[10., 11., 12.,  0.],
          [13., 14., 15.,  0.],
          [16., 17., 18.,  0.],
          [ 0.,  0.,  0.,  0.]]]])
torch.Size([48])
tensor([ 0.,  1.,  0.,  0.,  1.,  2.,  1.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,
         0.,  0.,  1.,  2.,  1.,  0.,  0.,  3.,  3.,  0.,  0.,  1., 10.,  0.,
         0.,  0.,  0.,  0., 10., 11., 12.,  0., 13., 14., 15.,  0., 16., 17.,
        18.,  0.,  0.,  0.,  0.,  0.])
torch.Size([1, 48])
tensor([[ 0.,  1.,  0.,  0.,  1.,  2.,  1.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,
          0.,  0.,  1.,  2.,  1.,  0.,  0.,  3.,  3.,  0.,  0.,  1., 10.,  0.,
          0.,  0.,  0.,  0., 10., 11., 12.,  0., 13., 14., 15.,  0., 16., 17.,
         18.,  0.,  0.,  0.,  0.,  0.